In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import sys
from pathlib import Path

# Add the parent directory to sys.path
sys.path.append(str(Path().resolve().parent))

# Now import the module
from normalization.conditions import assign_equations, volumen

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_original = pd.read_csv('../csvs/ciclo_1.csv', low_memory=False)
modelos = pd.read_excel('../csvs/modelos.xlsx')
df_regression = pd.read_csv('../linear_regression/ciclo1_out.csv',low_memory=False)
print("All the files where opened correctly")


All the files where opened correctly


In [5]:
n = len(df_original)
is_predicted = np.zeros(n)
df_original.insert(16, 'is_predicted', is_predicted)
print("Insertion correctly")

Insertion correctly


In [6]:
index = df_regression.index
df_original.loc[index] = df_regression
print("The original was changed for regression data")

The original was changed for regression data


In [7]:
df_original.loc[(df_original.condicion == "Tocón") & (df_original.altura <= 0) , "altura"] = .3
df_original.loc[(df_original.condicion == "Tocón") & (df_original.altura.isnull()), "altura"] = .3

df_original.loc[(df_original.condicion == "Tocón") & (df_original.diametro <= 0), "diametro"] = 7.5
df_original.loc[(df_original.condicion == "Tocón") & (df_original.diametro.isnull()), "diametro"] = .3


df_original.loc[(df_original.condicion != "Tocón") & (df_original.diametro.isnull()), "diametro"] = 7.5
df_original.loc[(df_original.condicion != "Tocón") & (df_original.altura.isnull()), "altura"] = 1.3

#b_eq = np.empty(n, dtype=object)


In [8]:
df_original

,id,anio_levantamiento,conglomerado,sitio,condicion,especie_id,familia,genero,epiteto,categoria_infra,infraespecie,numero_arbol,numero_tallo,tallos,diametro,altura,is_predicted,latitud,longitud,clave_ecoregion_n2,clave_bur,referencia_1,referencia_2,user_id,created_at,updated_at,grado_putrefaccion,clave_ecoregion_n1,clave_ecoregion_n3,clave_ecoregion_n4,estado_inegi,estado,edad
0,1,2005,60,1,Vivo,680.0,Ericaceae,Arctostaphylos,NaN,NaN,NaN,1,NaN,3.0,8.3,3.20,0.0,32.515597,-116.350450,11.1,MXL,caracteristica_vegetacion_mayor,107497,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.1,11.1.1.3,2,Baja California,NaN
1,2,2005,64,4,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,1,NaN,0.0,14.0,5.50,0.0,32.532903,-116.140585,11.1,MXL,caracteristica_vegetacion_mayor,101063,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.1,11.1.1.3,2,Baja California,NaN
2,3,2005,153,1,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,1,NaN,NaN,17.0,5.50,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,1,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,74.0
3,4,2005,153,1,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,2,NaN,NaN,14.5,5.85,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,2,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,61.0
4,5,2005,153,1,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,3,NaN,NaN,21.0,6.25,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,3,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305125,1305126,2005,78002,4,Vivo,19494.0,Rhizophoraceae,Rhizophora,mangle,NaN,NaN,6,NaN,NaN,31.5,20.00,0.0,14.587356,-92.270686,15.6,VHL,caracteristica_arbolado,1267537,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,15,15.6.1,15.6.1.1,7,Chiapas,NaN
1305126,1305127,2005,78002,4,Vivo,19494.0,Rhizophoraceae,Rhizophora,mangle,NaN,NaN,7,NaN,NaN,35.3,20.00,0.0,14.587356,-92.270686,15.6,VHL,caracteristica_arbolado,1267536,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,15,15.6.1,15.6.1.1,7,Chiapas,NaN
1305127,1305128,2005,78002,4,Vivo,19494.0,Rhizophoraceae,Rhizophora,mangle,NaN,NaN,8,NaN,NaN,30.0,19.00,0.0,14.587356,-92.270686,15.6,VHL,caracteristica_arbolado,1267535,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,15,15.6.1,15.6.1.1,7,Chiapas,NaN
1305128,1305129,2005,78002,4,Vivo,19494.0,Rhizophoraceae,Rhizophora,mangle,NaN,NaN,9,NaN,NaN,30.0,20.00,0.0,14.587356,-92.270686,15.6,VHL,caracteristica_arbolado,1267533,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,15,15.6.1,15.6.1.1,7,Chiapas,NaN


In [9]:
df_original.iloc[:10]

,id,anio_levantamiento,conglomerado,sitio,condicion,especie_id,familia,genero,epiteto,categoria_infra,infraespecie,numero_arbol,numero_tallo,tallos,diametro,altura,is_predicted,latitud,longitud,clave_ecoregion_n2,clave_bur,referencia_1,referencia_2,user_id,created_at,updated_at,grado_putrefaccion,clave_ecoregion_n1,clave_ecoregion_n3,clave_ecoregion_n4,estado_inegi,estado,edad
0,1,2005,60,1,Vivo,680.0,Ericaceae,Arctostaphylos,NaN,NaN,NaN,1,NaN,3.0,8.300000,3.20,0.0,32.515597,-116.350450,11.1,MXL,caracteristica_vegetacion_mayor,107497,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.1,11.1.1.3,2,Baja California,NaN
1,2,2005,64,4,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,1,NaN,0.0,14.000000,5.50,0.0,32.532903,-116.140585,11.1,MXL,caracteristica_vegetacion_mayor,101063,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.1,11.1.1.3,2,Baja California,NaN
2,3,2005,153,1,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,1,NaN,NaN,17.000000,5.50,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,1,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,74.0
3,4,2005,153,1,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,2,NaN,NaN,14.500000,5.85,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,2,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,61.0
4,5,2005,153,1,Vivo,26426.0,Pinaceae,Pinus,quadrifolia,NaN,NaN,3,NaN,NaN,21.000000,6.25,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,3,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN
5,6,2005,153,1,Vivo,7758.0,Cupressaceae,Juniperus,californica,NaN,NaN,4,NaN,NaN,13.500000,3.20,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,4,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN
6,7,2005,153,1,Vivo,2270.0,Cupressaceae,Juniperus,NaN,NaN,NaN,5,NaN,NaN,13.500000,3.00,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,5,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN
7,8,2005,153,1,Vivo,2270.0,Cupressaceae,Juniperus,NaN,NaN,NaN,6,NaN,NaN,9.000000,2.90,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,6,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN
8,9,2005,153,1,Vivo,2270.0,Cupressaceae,Juniperus,NaN,NaN,NaN,7,NaN,NaN,9.500000,3.00,0.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,7,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN
9,10,2005,153,1,Vivo,2270.0,Cupressaceae,Juniperus,NaN,NaN,NaN,8,NaN,NaN,11.588285,3.50,2.0,32.448351,-116.077138,11.1,BCO,caracteristica_arbolado,8,1.0,2019-11-14 20:24:49.070,2019-11-14 20:24:49.070,NaN,11,11.1.3,11.1.3.1,2,Baja California,NaN


In [13]:
df_10 = df_original.iloc[:10000]

In [14]:
b_eq = assign_equations(df_10, modelos, variable_resultado="b", conditions="biomasa")

  1%|▎                                      | 86/10000 [00:00<00:23, 416.59it/s]

this is working and here is the indices [np.int64(117), np.int64(114), np.int64(114), np.int64(455)]
this is working and here is the indices [np.int64(271), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(272), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(271), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(272), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(

  2%|▋                                     | 188/10000 [00:00<00:21, 461.49it/s]

this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(272), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(272), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(170), np.int64(170), np.int64(170), np.int64(170)]
this is working and here is the indices [np.int64(170), np.int64(170), np.int64(170), np.int64(170)]
this is working and here is the indices [np.int64(170), np.int64(170), np.int64(170), np.int64(170)]
this is working and here is the indices [np.int64(170), np.int64(170), np.int64(170), np.int64(170)]
this is working and here is the indices [np.int64(272), np.int64(412), np.int64(272), np.int64(80)]
this is working and here is the indices [np.int64(170), np.int64(170), np.int64(170), np.int64(170)]
t

  2%|▉                                     | 235/10000 [00:00<00:22, 430.51it/s]

...

this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(339), np.int64(339), np.int64(402), np.int64(339)]
this is working and here is the indices [np.int64(287), np.int64(87), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(112), np.int64(112), np.int64(112), np.int64

 99%|████████████████████████████████████▋| 9930/10000 [00:20<00:00, 497.90it/s]

this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is th

100%|████████████████████████████████████| 10000/10000 [00:20<00:00, 481.10it/s]

this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is the indices [np.int64(24), np.int64(24), np.int64(63), np.int64(87)]
this is working and here is th

In [16]:
len(b_eq)

10000

In [19]:
b_eq

array(['(GE*Exp((-1.349)+(1.98*LOG(([d130])))+ (0.207*(LOG(([d130])**2))+(-0.0281*(LOG(([d130])**3))))))',
       '(Exp(-3.139)*[d130]**2.585)', '(Exp(-2.818)*[d130]**2.574)', ...,
       'Exp(-2.27)*([d130]**2.39)', '(Exp(-3.416)*[d130]**2.715)',
       '(Exp(-2.084)*[d130]**2.323)'], dtype=object)

In [23]:
(b_eq['grado_putrefaccion']).idxmin()

/tmp/ipykernel_14778/303704464.py:1: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  (b_eq['grado_putrefaccion']).idxmin()


nan

In [ ]:
def count_variables(expression):
    potential_vars = re.findall(r'\b[a-zA-Z_]\w*\b', expression)
    functions_and_constants = {'Exp', 'LN'}
    variables = [var for var in potential_vars if var not in functions_and_constants]
    unique_variables = set(variables)
    return len(unique_variables)

variable_counts = b_eq['ecuacion'].apply(count_variables)

# Find the index of the row with the maximum number of variables
max_variable_index = variable_counts.idxmax()


rango = (b_eq['diametro_max'] - b_eq['diametro_min']).idxmin()
radio = (b_eq['r2']).idxmax()
arboles = (b_eq['numero_arboles']).idxmax()
variables = max_variable_index
if not np.isnan(rango):
    return b_eq.loc[rango]
elif not np.isnan(radio):
    return b_eq.loc[radio]
elif not np.isnan(arboles):
    return b_eq.loc[arboles]
elif not np.isnan(variables):
    return b_eq.iloc[variables]
else:
    return "This does not have eq that match the conditions"

In [48]:
def find_max_criteria_row(df):
    """
    Finds the row in the DataFrame `df` that satisfies the maximum or minimum criteria for 
    several specific columns. It checks for the following criteria in order:
    
    1. The minimum difference between 'diametro_max' and 'diametro_min'.
    2. The maximum value in the 'r2' column.
    3. The maximum value in the 'numero_arboles' column.
    4. The maximum number of unique variables in the 'ecuacion' column.
    
    The function returns the first row that satisfies any of these criteria.

    Parameters:
    ----------
    df : pandas.DataFrame
        The DataFrame containing the columns 'diametro_max', 'diametro_min', 'r2', 'numero_arboles', 
        and 'ecuacion'.

    Returns:
    -------
    pandas.Series or str
        The row from the DataFrame that meets the first of the specified criteria.
        If no criteria are met, it returns a message indicating no matches.
    """

    def count_variables(expression):
        """
        Counts the number of unique variables in a mathematical expression.

        Parameters:
        ----------
        expression : str
            The mathematical expression as a string.

        Returns:
        -------
        int
            The number of unique variables in the expression.
        """
        potential_vars = re.findall(r'\b[a-zA-Z_]\w*\b', expression)
        functions_and_constants = {'Exp', 'LN'}
        variables = [var for var in potential_vars if var not in functions_and_constants]
        unique_variables = set(variables)
        return len(unique_variables)

    # Calculate variable counts for each row in the 'ecuacion' column
    variable_counts = df['ecuacion'].apply(count_variables)

    # Find indices for the specified criteria
    indices = [
        (df['diametro_max'] - df['diametro_min']).idxmin(),  # Minimum range
        df['r2'].idxmax(),                                   # Maximum r2
        df['numero_arboles'].idxmax(),                       # Maximum number of trees
        variable_counts.idxmax()                             # Maximum variables
    ]

    # Iterate over indices and return the first valid row
    for index in indices:
        if not np.isnan(index):
            return df.loc[index]

    # If no valid index is found, return a message
    return "This does not have eq that match the conditions"

In [49]:
# Example usage with a DataFrame b_eq
result = find_max_criteria_row(b_eq)
print(result)

id                                                                  118
id_anterior                                                       112.0
variable_resultado                                                    b
tipo                                                            General
ecuacion              (GE*Exp((-1.349)+(1.98*LN((d130)))+ (0.207*(LN...
ecuacion_php          (GE*Exp((-1.349)+(1.98*LOG(([d130])))+ (0.207*...
especie_id                                                          NaN
familia                                                             NaN
genero                                                              NaN
epiteto                                                             NaN
condicion                                                          Vivo
componente                                                Todo el árbol
diametro_min                                                        5.0
diametro_max                                                    

In [ ]:
#c_eq = assign_equations(df_original, modelos, variable_resultado="c", conditions="carbono")
#p_eq = assign_equations(df_original, modelos, variable_resultado="p", conditions="densidad")

#df_original_muerto = df_original[df_original.condicion == "Muertos"]
#df_original_tocon = df_original[df_original.condicion == "Tocón"]

#v_eq = volumen(df_original_muerto, df_original_tocon, modelos)


np.save('b_eq.npy', b_eq)